## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/135bert-base-best/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_1.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_2.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_3.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_4.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_5.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=False)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.928149,0.609735,0.221895,0.540104,0.572166,0.572918,0.652510,0.636687,0.598679,...,0.888008,0.894837,0.541011,0.948660,0.948171,0.762855,0.093198,0.063222,0.767874,0.911540
1,46,0.876532,0.509205,0.003897,0.814316,0.799247,0.931776,0.575038,0.488079,0.046395,...,0.705215,0.948260,0.671089,0.968573,0.981133,0.874159,0.863535,0.099655,0.122555,0.891022
2,70,0.921077,0.649378,0.023096,0.797493,0.866353,0.926540,0.604773,0.524976,0.180634,...,0.841238,0.943780,0.621225,0.971554,0.977915,0.867791,0.142476,0.067719,0.813825,0.918395
3,132,0.885716,0.432262,0.003566,0.748700,0.750232,0.913459,0.568154,0.425554,0.102467,...,0.726296,0.955832,0.672985,0.974808,0.987550,0.906491,0.820338,0.140983,0.458128,0.920555
4,200,0.919363,0.481965,0.036397,0.834924,0.702791,0.824109,0.650676,0.603689,0.076382,...,0.734050,0.919641,0.641398,0.956973,0.962038,0.839140,0.161834,0.097379,0.631973,0.905012


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.891398,0.591054,0.033752,0.747093,0.790623,0.851959,0.591164,0.497020,0.209960,...,0.806493,0.938507,0.665019,0.964549,0.975318,0.870626,0.496823,0.123417,0.528031,0.914052
std,2812.670060,0.044394,0.116579,0.062519,0.091763,0.102072,0.120897,0.049039,0.084535,0.162577,...,0.076088,0.019869,0.047037,0.011402,0.010402,0.035430,0.302354,0.058297,0.252487,0.018282
min,39.000000,0.762012,0.378027,0.001677,0.313048,0.350311,0.286206,0.498254,0.367263,0.006010,...,0.602302,0.861694,0.541011,0.917587,0.915652,0.761019,0.006842,0.004627,0.049842,0.844521
25%,2572.000000,0.861991,0.488682,0.005976,0.696197,0.746607,0.833465,0.552597,0.433452,0.085372,...,0.745205,0.927340,0.629778,0.957725,0.970279,0.847469,0.182084,0.083587,0.318622,0.901948
50%,5093.000000,0.892828,0.591529,0.009730,0.753196,0.803385,0.897466,0.580343,0.471786,0.151269,...,0.810237,0.941887,0.661236,0.965604,0.977676,0.874323,0.549041,0.126457,0.511817,0.914862
75%,7482.000000,0.927339,0.676948,0.026177,0.802911,0.859435,0.925262,0.620994,0.542500,0.297331,...,0.869520,0.953304,0.696761,0.972962,0.982546,0.895309,0.770300,0.164679,0.738240,0.928101
max,9640.000000,0.977726,0.906759,0.558249,0.960586,0.973923,0.972487,0.741710,0.782489,0.759965,...,0.961673,0.981698,0.822819,0.989520,0.993359,0.962978,0.951156,0.304418,0.986529,0.960184
